### Maximal Marginal Relevance
MMR (Maximal Marginal Relevance) is a powerful diversity-aware retrieval technique used in information retrieval and RAG pipelines to balance relevance and novelty when selecting documents.

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


In [4]:
#Step 1: Load and chunk the document
loader = TextLoader("langchain_rag_dataset.txt")
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)
chunks

[Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain is an open-source framework designed to simplify the development of applications using large language models (LLMs).\nLangChain provides abstractions for working with prompts, chains, memory, and agents, making it easier to build complex LLM-based systems.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='The framework supports integration with various vector databases like FAISS and Chroma for semantic retrieval.\nLangChain enables Retrieval-Augmented Generation (RAG) by allowing developers to fetch relevant context before generating responses.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
 Document(metadata={'

In [6]:
## Step 2 : FIASS vector store with Hugging Face embeddings 

embedding_model = HuggingFaceEmbeddings(model = "all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding_model)

In [7]:
### Step 3: Create MMR Retriever 
retriever = vectorstore.as_retriever(
    search_type = "mmr", 
    seatch_kwargs = {"k":3}

)

In [ ]:
### Step 4: Prompt and LLM

prompt = PromptTemplate.from_template("""
Answer the question based on the context provided.
Context:
{context}

Question:{input}
""")

llm = init_chat_model("groq:gemma2-9b-it")

In [10]:
### step 5: RAG pipeline
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=document_chain)

In [13]:
#Step 6: Query 
query = {"input":"How does Langchain support agents and memory?"}
response = rag_chain.invoke(query)

In [15]:

print("Answer:\n", response['answer'])

Answer:
 LangChain supports agents in two key ways:

1. **Tool Integration:** Agents can leverage external tools like calculators, search APIs, or custom functions based on instructions. This allows them to perform tasks beyond simple text generation.
2. **Decision Making:** LangChain enables LLMs to act as agents, determining which tool to use and in what sequence to accomplish a given task.

Regarding memory, LangChain offers:

1. **Conversational Memory:** `ConversationBufferMemory` allows agents to retain previous interactions, leading to more coherent multi-turn conversations.
2. **Summarization Memory:** `ConversationSummaryMemory` provides a concise summary of past interactions, aiding in contextual understanding. 


Essentially, LangChain empowers agents with both the ability to access and utilize external tools and the memory necessary for making informed decisions and maintaining context within conversations. 



In [16]:
response

{'input': 'How does Langchain support agents and memory?',
 'context': [Document(id='41f13ae6-8ab8-42fd-bd94-493d344673e6', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
  Document(id='5a011857-5a48-467e-a2ad-200610bf8e38', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain agents can interact with external APIs and databases, enhancing the capabilities of LLM-powered applications.\nRAG pipelines in LangChain involve document loading, splitting, embedding, retrieval, and LLM-based response generation.'),
  Document(id='927dd9e9-ba5e-4f31-9751-05bb644bde4d', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain allows LLMs to act as agents that decide which tool to call and in what order duri